In [13]:
import numpy as np
import pandas as pd
import pickle
from ipywidgets import interact
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
!pip install --upgrade pip
!pip3 install pickle5

     |████████████████████████████████| 1.7MB 373kB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1
     |████████████████████████████████| 124 kB 224 kB/s            


### J'ai des pb avec lire les pickles sur mon google drive donc je laisse de téco pr l'instant


In [14]:
#df0 = pd.read_parquet('pass0.parquet')
#df1 = pd.read_parquet('pass1.parquet') #kanser, mais change ces lignes
import pickle5 as pickle
with open('/Users/hectordevie/Desktop/DTU/Deep Learning/Projet/pass0_cleaned.pickle', 'rb') as handle:
    df0 = pickle.load(handle)
with open('/Users/hectordevie/Desktop/DTU/Deep Learning/Projet/pass1_cleaned.pickle', 'rb') as handle:
    df1 = pickle.load(handle)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
print(list(df0))
df0.head()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [17]:
def show(col):
    try:
        plt.hist(df0[col].map(len), bins=20)
        plt.show()
        plt.plot(df0.loc[0,col])
    except:
        ...#reeee
    
interact(show, col=list(df0));

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
s0 = df0.loc[0]
mask = s0['GM.T'] == 'acc.xyz'
accT = s0['GM.TS_or_Distance'][mask]
accZ = s0['GM.acc.xyz.z']
obdT = s0['GM.TS_or_Distance'][~mask]
obd = s0['GM.obd.spd_veh.value']

# plt.plot(accZ)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(accT, accZ, color='#1f77b4')
ax2.plot(obdT, obd, color='#ff7f0e');

# weird stuff on the orange curve (?)

In [ ]:
from torch.utils.data import Dataset

class MyDS(Dataset):
    def __init__(self, fname, size=None):
        df = fname
        zacc = df['GM.acc.xyz.z'] # Series[arr[:]float]
        
        if not size:
            size = zacc.map(len).quantile(.1)
            size = (int(size) // 32) * 32
        self.sz = size
        zacc = zacc[zacc.map(len) >= size]
        
        def truncate(arr):
            return arr[:size]
        zacc = np.stack(zacc.map(truncate)).astype('float32')
        # print(zacc.shape, size)
        self.zacc = zacc
        
    
    def __getitem__(self, idx):
        return self.zacc[idx][None,:]
    
    def __len__(self):
        return len(self.zacc)

L = 2048
my_data = MyDS(df0, 2048) #size = N

In [ ]:
import torch
import torch.nn as nn 

class AE(nn.Module):
    def __init__(self, L):
        super().__init__()
        
        mid = L // (4*4*4) #latent space dims
        assert L % (4*4*4) == 0
        
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 3, 5, padding='same'),
            nn.MaxPool1d(4,4),
            nn.ReLU(),
            nn.Conv1d(3, 3, 5, padding='same'),
            nn.MaxPool1d(4,4),
            nn.ReLU(),
            nn.Conv1d(3, 3, 5, padding='same'),
            nn.MaxPool1d(4,4),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(mid*3, 10)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(10, 3*mid),
            nn.Unflatten(1, (3,mid)),
            nn.ReLU(),
            nn.Upsample(scale_factor=4),
            nn.Conv1d(3, 3, 5, padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=4),
            nn.Conv1d(3, 3, 5, padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=4),
            nn.Conv1d(3, 1, 5, padding='same'),
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        # print(encoded.shape)
        decoded = self.decoder(encoded)
        return decoded
        
    def get_latent(self,x):
      return self.encoder(x)

In [ ]:
def visualization(x,display = True):
      tsne = TSNE(n_components=2, n_iter=3000, )
      vis = tsne.fit_transform(x)
      if display:
        fig, ax = plt.subplots()
        ax.scatter(*vis.T)
        plt.show()

        #df_subset['tsne-2d-one'] = tsne_results[:,0]
        #df_subset['tsne-2d-two'] = tsne_results[:,1]
        #plt.figure(figsize=(16,10))
        #sns.scatterplot(
        #x="tsne-2d-one", y="tsne-2d-two",
        #hue="y",
        #palette=sns.color_palette("hls", 10),
        #data=df_subset,
        #legend="full",
        #alpha=0.3)
      else:
        return vis

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(my_data, shuffle=True)
model = AE(L)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


for x in dataloader:
    
    pred = model(x)
    loss = loss_fn(pred, x)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    #print(float(loss))

In [ ]:
latent = []

for x in dataloader:
  latent.append(model.get_latent(x).detach().numpy()[0])
latent
visualization(latent)